In [ ]:
!pip install transformers==3
!pip install cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.8.1-cp37-cp37m-linux_x86_64.whl
import numpy as np
import pandas as pd
!pip install fastapi
!pip install pyngrok
!pip install uvicorn

from fastapi import FastAPI
import nest_asyncio
from pyngrok import ngrok
import uvicorn
from transformers import RobertaModel, RobertaTokenizer
from transformers import AutoConfig, AutoModelForSequenceClassification, AutoTokenizer
from keras.utils.np_utils import to_categorical
# from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences as pad

import torch_xla.core.xla_model as xm

EPOCHS = 20
SPLIT = 0.8
MAXLEN = 48
DROP_RATE = 0.3
np.random.seed(42)

OUTPUT_UNITS = 3
BATCH_SIZE = 384
LR = (4e-5, 1e-2)
ROBERTA_UNITS = 768
VAL_BATCH_SIZE = 384
MODEL_SAVE_PATH = 'sentiment_model.pt'
model = 'roberta-base'

Google Drive was used to mount and get sentiment_model.pt file of the previous running algorithm. Since model file was too big and it takes much time to upload we have tested code one time and we have commented out now because of separate drives

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import torch.nn as nn
import torch

class Roberta(nn.Module):
    def __init__(self):
        super(Roberta, self).__init__()
        self.softmax = nn.Softmax(dim=1)
        self.drop = nn.Dropout(DROP_RATE)
        
        self.roberta = RobertaModel.from_pretrained(model)
        self.dense = nn.Linear(ROBERTA_UNITS, OUTPUT_UNITS)
        
        
    def forward(self, inp, att):
        inp = inp.view(-1, MAXLEN)
        _, self.feat = self.roberta(inp, att)
        return self.softmax(self.dense(self.drop(self.feat)))

network = Roberta()
network.load_state_dict(torch.load('/content/drive/MyDrive/DSPD/dspd_sentiment_analysis.pt'))


<All keys matched successfully>

FAST API was deployed on google COLAB because of torch xla version, torch xla is not supported on local windows as we tried to integrate with local cpu settings.

In [9]:

app = FastAPI()


device = xm.xla_device()
network = network.to(device)
tokenizer =  AutoTokenizer.from_pretrained(model, use_fast=True)
def predict_sentiment(tweet):
    pg, tg = 'post', 'post'
    tweet_ids = tokenizer.encode(tweet.strip())
    sent = {0: 'positive', 1: 'neutral', 2: 'negative'}

    att_mask_idx = len(tweet_ids) - 1
    if 0 not in tweet_ids: tweet_ids = 0 + tweet_ids
    tweet_ids = pad([tweet_ids], maxlen=MAXLEN, value=1, padding=pg, truncating=tg)
    # device = xm.xla_device()
    att_mask = np.zeros(MAXLEN)
    att_mask[1:att_mask_idx] = 1
    att_mask = att_mask.reshape((1, -1))
    if 2 not in tweet_ids: tweet_ids[-1], att_mask[-1] = 2, 0
    tweet_ids, att_mask = torch.LongTensor(tweet_ids), torch.LongTensor(att_mask)
    return sent[np.argmax(network.forward(tweet_ids.to(device), att_mask.to(device)).detach().cpu().numpy())]

#Default sentiment
@app.get("/")
def read_root():
  return predict_sentiment("I dont like this")

#parametric input i.e *.io/i like this
@app.get("/{sentiment}")
def predict_Sentiment(sentiment: str):
  vara = predict_sentiment(sentiment)
  return vara

ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)

Public URL: http://36aed55be163.ngrok.io


INFO:     Started server process [58]
INFO:uvicorn.error:Started server process [58]
INFO:     Waiting for application startup.
INFO:uvicorn.error:Waiting for application startup.
INFO:     Application startup complete.
INFO:uvicorn.error:Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
INFO:uvicorn.error:Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     175.107.213.39:0 - "GET / HTTP/1.1" 200 OK
INFO:     175.107.213.39:0 - "GET /favicon.ico HTTP/1.1" 200 OK
INFO:     175.107.213.39:0 - "GET /I%20want%20to%20know%20more%20about%20your%20product. HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:uvicorn.error:Shutting down
INFO:     Waiting for application shutdown.
INFO:uvicorn.error:Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:uvicorn.error:Application shutdown complete.
INFO:     Finished server process [58]
INFO:uvicorn.error:Finished server process [58]
